In [33]:
import pandas as pd

df = pd.read_csv ('data/nutrition_table_balanced.csv', encoding= 'unicode_escape')
print (df)

                                          Product Name  \
0         Kellogg's Rice Krispies Breakfast Cereal 1kg   
1       Sainsbury's Berry Hooplas, Summer Edition 375g   
2    Sainsbury's  Peach Melba Granola, Summer Editi...   
3    Kellogg's Special K Original Breakfast Cereal ...   
4           Kellogg's Crunchy Nut Breakfast Cereal 1kg   
..                                                 ...   
594  Doritos Stax Sour Cream & Onion Sharing Snacks...   
595                  Walkers Prawn Cocktail Crisps 45g   
596  Pringles Sizzl'N Spicy Chorizo Flavour Sharing...   
597  Doritos Flame Grilled Steak Sharing Tortilla C...   
598  Walkers Max KFC Kentucky Fried Chicken Sharing...   

                                   Product Description Category  Energy  \
0    Toasted Rice Cereal Fortified with Vitamins an...   cereal   389.0   
1    Coated cereal extrudate with natural strawberr...   cereal   395.0   
2    A blend of honey toasted wholegrain oats with ...   cereal   437.0   
3  

In [34]:
import torch
import numpy as np
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {'cereal':0,
          'chocolate':1,
         'rice':2,
         'yogurts':3,
         'crisps':4}

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):
        self.labels = [labels[label] for label in df['Category']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['Product Name']]


    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [35]:
'''build bert model'''

from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 5)
        self.relu = nn.Softmax()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output, dim=1)

        return final_layer

In [36]:
np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), 
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))

479 60 60


In [54]:
from torch.optim import Adam
from tqdm import tqdm
import torch

def train(model, train_data, val_data, learning_rate, epochs):
    
    #train_data = train_data.type(torch.LongTensor)

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  
EPOCHS = 9
model = BertClassifier(dropout=0.1)
LR = 5.00E-07
              
train(model, df_train, df_val, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|                                                                                           | 0/240 [00:00<?, ?it/s]/tmp/ipykernel_155/1648410969.py:22: 

Epochs: 1 | Train Loss:  0.802                 | Train Accuracy:  0.217                 | Val Loss:  0.791                 | Val Accuracy:  0.250


100%|█████████████████████████████████████████████████████████████████████████████████| 240/240 [08:58<00:00,  2.24s/it]


Epochs: 2 | Train Loss:  0.774                 | Train Accuracy:  0.428                 | Val Loss:  0.757                 | Val Accuracy:  0.533


100%|█████████████████████████████████████████████████████████████████████████████████| 240/240 [08:53<00:00,  2.22s/it]


...

Epochs: 7 | Train Loss:  0.598                 | Train Accuracy:  0.900                 | Val Loss:  0.595                 | Val Accuracy:  0.917


100%|█████████████████████████████████████████████████████████████████████████████████| 240/240 [08:21<00:00,  2.09s/it]


Epochs: 8 | Train Loss:  0.573                 | Train Accuracy:  0.925                 | Val Loss:  0.574                 | Val Accuracy:  0.917


100%|█████████████████████████████████████████████████████████████████████████████████| 240/240 [08:27<00:00,  2.11s/it]


Epochs: 9 | Train Loss:  0.552                 | Train Accuracy:  0.973                 | Val Loss:  0.556                 | Val Accuracy:  0.933


In [55]:
df_db = pd.read_csv('data/Sample_db.csv')

In [56]:
"""
Multi-class evaluator
"""
from sklearn.metrics import f1_score
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    TP = 0
    FP = 0
    FN = 0
    with torch.no_grad():
        y_pred = []
        y_true = []
        for test_input, test_label in test_dataloader:
            test_label = test_label.to(device)
            mask = test_input['attention_mask'].to(device)
            input_id = test_input['input_ids'].squeeze(1).to(device)

            i = 0
            output = model(input_id, mask)
            print(output)
            selected_output = output.argmax(dim=1)[i]#0,1,2,3,4
            y_pred.append(selected_output.detach().numpy())
            y_true.append(test_label[0].detach().numpy())

            acc = (output.argmax(dim=1) == test_label).sum().item()
            total_acc_test += acc
            
            #selected = (output.argmax(dim=1) == 1).sum().item()

    '''
    Weighted F1: 
    '''
    print(y_true)
    print(y_pred)
    weighted_f1 = f1_score(y_true, y_pred, average='weighted')
    #F1 = TP/(TP + 0.5*(FP+FN))
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    print(f'Test Weighted F1 score: {weighted_f1: .3f}')
    return (total_acc_test / len(test_data), weighted_f1)
    
evaluate(model, df_db)

/tmp/ipykernel_155/1648410969.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_layer = self.relu(linear_output)


tensor([[0.0511, 0.0368, 0.3369, 0.0928, 0.4824],
        [0.0457, 0.0299, 0.2440, 0.0589, 0.6215]])
tensor([[0.0122, 0.0137, 0.0195, 0.0224, 0.9322],
        [0.0120, 0.0091, 0.0173, 0.0233, 0.9384]])
tensor([[0.1037, 0.0675, 0.1852, 0.5911, 0.0525],
        [0.1052, 0.0779, 0.0934, 0.6810, 0.0425]])
tensor([[0.0098, 0.0096, 0.0132, 0.0123, 0.9552],
        [0.0138, 0.0112, 0.0127, 0.0164, 0.9460]])
tensor([[0.0133, 0.0129, 0.0137, 0.0179, 0.9422],
        [0.0623, 0.0593, 0.6081, 0.1699, 0.1003]])
tensor([[0.0303, 0.0552, 0.7744, 0.0737, 0.0664],
        [0.3024, 0.2093, 0.1875, 0.2569, 0.0440]])
tensor([[0.0988, 0.0699, 0.3257, 0.3743, 0.1313],
        [0.0485, 0.0413, 0.7814, 0.0528, 0.0761]])
tensor([[0.1003, 0.0545, 0.4203, 0.3238, 0.1011],
        [0.2957, 0.2346, 0.1619, 0.2643, 0.0435]])
tensor([[0.4369, 0.1807, 0.0928, 0.2267, 0.0629],
        [0.1656, 0.0851, 0.1320, 0.5402, 0.0771]])
tensor([[0.1001, 0.0515, 0.2371, 0.3344, 0.2769],
        [0.1898, 0.0800, 0.1567, 0.4481, 

(0.44680851063829785, 0.6386452241715399)

In [49]:
##split
import random
from csv import writer

np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), 
                                     [int(.8*len(df)), int(.9*len(df))])
print(len(df_train),len(df_val), len(df_test))

EPOCHS = 5
LR = 1e-6

for i in range(200):
    EPOCHS = random.randint(2,15)
    LR = 0.5 * random.randint(1,2) * 10 ** (-random.randint(1,10))
    DROPOUT = 0.1 * random.randint(1,9)
    model = BertClassifier(dropout=DROPOUT)           
    train(model, df_train, df_val, LR, EPOCHS)
    acc, f1 = evaluate(model, df_test)
    print(f'EPOCHS: {EPOCHS} | LR:{LR} | DROPOUT:{DROPOUT}')
    list_data = [EPOCHS,LR,DROPOUT,acc,f1]
    with open('data/training_category.csv','a', newline='') as f_object:
        writer_object = writer(f_object)
        writer_object.writerow(list_data)
        f_object.close()

479 60 60


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|                                                                                           | 0/240 [00:00<?, ?it/s]/tmp/ipykernel_155/1648410969.py:22: 

Epochs: 1 | Train Loss:  0.807                 | Train Accuracy:  0.225                 | Val Loss:  0.807                 | Val Accuracy:  0.183


100%|█████████████████████████████████████████████████████████████████████████████████| 240/240 [08:54<00:00,  2.23s/it]


Epochs: 2 | Train Loss:  0.807                 | Train Accuracy:  0.221                 | Val Loss:  0.807                 | Val Accuracy:  0.217


...

Epochs: 10 | Train Loss:  0.841                 | Train Accuracy:  0.200                 | Val Loss:  0.844                 | Val Accuracy:  0.217


100%|█████████████████████████████████████████████████████████████████████████████████| 240/240 [09:13<00:00,  2.31s/it]


Epochs: 11 | Train Loss:  0.853                 | Train Accuracy:  0.186                 | Val Loss:  0.870                 | Val Accuracy:  0.150
tensor([[7.1878e-06, 1.6004e-03, 9.9787e-01, 4.4752e-05, 4.7538e-04],
        [3.0621e-02, 5.0514e-03, 9.4801e-01, 5.3109e-04, 1.5787e-02]])
tensor([[2.4235e-06, 7.2856e-05, 9.9992e-01, 4.2740e-06, 1.5519e-07],
        [2.3811e-08, 2.6640e-06, 9.9998e-01, 6.3836e-07, 1.2675e-05]])
tensor([[2.4035e-02, 6.7508e-03, 7.7814e-01, 3.0694e-04, 1.9076e-01],
        [6.3461e-01, 2.3774e-03, 3.5966e-01, 3.0598e-04, 3.0455e-03]])
tensor([[3.4390e-02, 1.8507e-03, 7.5988e-01, 9.4811e-05, 2.0379e-01],
        [2.3683e-05, 1.6324e-08, 9.9974e-01, 8.3355e-07, 2.3476e-04]])
tensor([[7.8527e-03, 1.2255e-05, 9.8942e-01, 2.1016e-05, 2.6971e-03],
        [4.8760e-04, 1.1843e-03, 9.9787e-01, 4.4642e-04, 9.6985e-06]])
tensor([[6.0508e-04, 6.4222e-03, 9.6458e-01, 1.5527e-03, 2.6837e-02],
        [6.6404e-09, 1.8869e-06, 9.9975e-01, 9.0409e-11, 2.4756e-04]])
tensor(

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|                                                                                           | 0/240 [00:00<?, ?it/s]/tmp/ipykernel_155/1648410969.py:22: 

KeyboardInterrupt: 